# **Bioinformatics Project - Computational Drug Discovery [Part 1] Download Bioactivity Data**

In this Jupyter notebook, we will be building a real-life **data science project** that you can include in your **data science portfolio**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 1**, we will be performing Data Collection and Pre-Processing from the ChEMBL Database.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 16.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for Monoamine Oxidase B**

In [ ]:
# Target search for maob
target = new_client.target
target_query = target.search('maob')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P27338', 'xref_name': None, 'xre...",Homo sapiens,Monoamine oxidase B,15.0,False,CHEMBL2039,"[{'accession': 'P27338', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Mus musculus,Monoamine oxidase B,15.0,False,CHEMBL3050,"[{'accession': 'Q8BW75', 'component_descriptio...",SINGLE PROTEIN,10090
2,"[{'xref_id': 'P56560', 'xref_name': None, 'xre...",Bos taurus,Monoamine oxidase B,15.0,False,CHEMBL2756,"[{'accession': 'P56560', 'component_descriptio...",SINGLE PROTEIN,9913
3,"[{'xref_id': 'P19643', 'xref_name': None, 'xre...",Rattus norvegicus,Monoamine oxidase B,15.0,False,CHEMBL2993,"[{'accession': 'P19643', 'component_descriptio...",SINGLE PROTEIN,10116
4,[],Rattus norvegicus,Monoamine oxidase,13.0,False,CHEMBL2095196,"[{'accession': 'P19643', 'component_descriptio...",PROTEIN FAMILY,10116
5,[],Homo sapiens,Monoamine oxidase,13.0,False,CHEMBL2095205,"[{'accession': 'P21397', 'component_descriptio...",PROTEIN FAMILY,9606
6,[],Mus musculus,Monoamine oxidase,13.0,False,CHEMBL2111442,"[{'accession': 'Q8BW75', 'component_descriptio...",PROTEIN FAMILY,10090
7,[],Bos taurus,Monoamine oxidase,13.0,False,CHEMBL2111399,"[{'accession': 'P56560', 'component_descriptio...",PROTEIN FAMILY,9913


### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

We will assign the fifth entry (which corresponds to the target protein, *coronavirus 3C-like proteinase*) to the ***selected_target*** variable 

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL2039'

Here, we will retrieve only bioactivity data for *coronavirus 3C-like proteinase* (CHEMBL3927) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,184069,[],CHEMBL706869,Compound was tested for inhibition of monoamin...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NCc3ccccc3)c(Br)c21,Outside typical range,Values for this activity type are unusually la...,CHEMBL1151505,Bioorg. Med. Chem. Lett.,1996.0,None,CHEMBL156630,None,CHEMBL156630,None,False,http://www.openphacts.org/units/Nanomolar,308504,>,1,True,>,None,IC50,nM,None,1000000.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,uM,UO_0000065,None,1000.0
1,None,185443,[],CHEMBL706869,Compound was tested for inhibition of monoamin...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NC)c(Cl)c21,Outside typical range,Values for this activity type are unusually la...,CHEMBL1151505,Bioorg. Med. Chem. Lett.,1996.0,None,CHEMBL155754,None,CHEMBL155754,None,False,http://www.openphacts.org/units/Nanomolar,308506,>,1,True,>,None,IC50,nM,None,1000000.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,uM,UO_0000065,None,1000.0
2,None,185700,[],CHEMBL734179,Inhibitory concentration against Monoamine oxi...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,None,None,CHEMBL1136363,Bioorg. Med. Chem. Lett.,2003.0,"{'bei': '23.09', 'le': '0.42', 'lle': '5.04', ...",CHEMBL350093,None,CHEMBL350093,7.75,False,http://www.openphacts.org/units/Nanomolar,319607,=,1,True,=,None,IC50,nM,None,18.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,nM,UO_0000065,None,18.0


Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.to_csv('/content/drive/MyDrive/yassir/bioactivity_data_raw.csv', index=False)

## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [ ]:
df2 = df[df.standard_value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,184069,[],CHEMBL706869,Compound was tested for inhibition of monoamin...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NCc3ccccc3)c(Br)c21,Outside typical range,Values for this activity type are unusually la...,CHEMBL1151505,Bioorg. Med. Chem. Lett.,1996.0,None,CHEMBL156630,None,CHEMBL156630,None,False,http://www.openphacts.org/units/Nanomolar,308504,>,1,True,>,None,IC50,nM,None,1000000.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,uM,UO_0000065,None,1000.0
1,None,185443,[],CHEMBL706869,Compound was tested for inhibition of monoamin...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NC)c(Cl)c21,Outside typical range,Values for this activity type are unusually la...,CHEMBL1151505,Bioorg. Med. Chem. Lett.,1996.0,None,CHEMBL155754,None,CHEMBL155754,None,False,http://www.openphacts.org/units/Nanomolar,308506,>,1,True,>,None,IC50,nM,None,1000000.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,uM,UO_0000065,None,1000.0
2,None,185700,[],CHEMBL734179,Inhibitory concentration against Monoamine oxi...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,None,None,CHEMBL1136363,Bioorg. Med. Chem. Lett.,2003.0,"{'bei': '23.09', 'le': '0.42', 'lle': '5.04', ...",CHEMBL350093,None,CHEMBL350093,7.75,False,http://www.openphacts.org/units/Nanomolar,319607,=,1,True,=,None,IC50,nM,None,18.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,nM,UO_0000065,None,18.0
3,None,187020,[],CHEMBL734178,Inhibitory concentration against Monoamine oxi...,B,None,None,BAO_0000190,BAO_0000357,single protein format,O=c1c(=O)c2ccc(OCCCC(F)(F)F)cc2c1=O,None,None,CHEMBL1136363,Bioorg. Med. Chem. Lett.,2003.0,"{'bei': '28.11', 'le': '0.55', 'lle': '6.53', ...",CHEMBL161907,None,CHEMBL161907,8.05,False,http://www.openphacts.org/units/Nanomolar,319604,=,1,True,=,None,IC50,nM,None,9.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,nM,UO_0000065,None,9.0
4,None,188355,[],CHEMBL734179,Inhibitory concentration against Monoamine oxi...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=S,None,None,CHEMBL1136363,Bioorg. Med. Chem. Lett.,2003.0,"{'bei': '24.77', 'le': '0.48', 'lle': '3.99', ...",CHEMBL17079,None,CHEMBL17079,8.36,False,http://www.openphacts.org/units/Nanomolar,319609,=,1,True,=,None,IC50,nM,None,4.4,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,nM,UO_0000065,None,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,None,20706656,[],CHEMBL4628301,Inhibition of recombinant human MAOB expressed...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,COC(=O)C(C)c1ccc(-c2ccc(NCc3nc(C)c(C)nc3C)cc2)...,None,None,CHEMBL4627303,Bioorg Med Chem,2020.0,"{'bei': '11.83', 'le': '0.22', 'lle': '-0.28',...",CHEMBL4636637,None,CHEMBL4636637,4.82,False,http://www.openphacts.org/units/Nanomolar,3487395,=,1,True,=,None,IC50,nM,None,15100.0,CHEMBL2039,Homo sapiens,Monoamine oxidase B,9606,None,None,IC50,uM,UO_0000065,None,15.1
4809,None,20706657,[],CHEMBL4628301,Inhibition of recombinant human MAOB expressed...,B,None,No

Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  #else:
  #  bioactivity_class.append("intermediate")

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL156630,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NCc3ccccc3)c(Br)c21,1000000.0
1,CHEMBL155754,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NC)c(Cl)c21,1000000.0
2,CHEMBL350093,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,18.0
3,CHEMBL161907,O=c1c(=O)c2ccc(OCCCC(F)(F)F)cc2c1=O,9.0
4,CHEMBL17079,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=S,4.4
...,...,...,...
4808,CHEMBL4636637,COC(=O)C(C)c1ccc(-c2ccc(NCc3nc(C)c(C)nc3C)cc2)...,15100.0
4809,CHEMBL887,C#CCN[C@@H]1CCc2ccccc21,11.6
4810,CHEMBL8706,C#CCN(C)CCCOc1ccc(Cl)cc1Cl,2340.0
4811,CHEMBL92401,CC(C)NNC(=O)c1ccncc1,2340.0


In [ ]:
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL156630,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NCc3ccccc3)c(Br)c21,1000000.0,inactive
1,CHEMBL155754,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NC)c(Cl)c21,1000000.0,inactive
2,CHEMBL350093,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,18.0,active
3,CHEMBL161907,O=c1c(=O)c2ccc(OCCCC(F)(F)F)cc2c1=O,9.0,active
4,CHEMBL17079,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=S,4.4,active
...,...,...,...,...
4808,CHEMBL4636637,COC(=O)C(C)c1ccc(-c2ccc(NCc3nc(C)c(C)nc3C)cc2)...,15100.0,NaN
4809,CHEMBL887,C#CCN[C@@H]1CCc2ccccc21,11.6,NaN
4810,CHEMBL8706,C#CCN(C)CCCOc1ccc(Cl)cc1Cl,2340.0,NaN
4811,CHEMBL92401,CC(C)NNC(=O)c1ccncc1,2340.0,NaN


Saves dataframe to CSV file

In [ ]:
df4.to_csv('/content/drive/MyDrive/yassir/bioactivity_data_preprocessed.csv', index=False)

In [ ]:
! ls -l

total 8
drwx------ 5 root root 4096 Dec  8 20:02 drive
drwxr-xr-x 1 root root 4096 Dec  3 14:33 sample_data


---